# This notebook will implement an optional pre-processing step that can be done prior to candidate label creation.
We will trim down a given quantity file by using associated qualifiers to keep only the most up-to-date value for time-series data. For example, if your quantity file includes the human development index (HDI) of various countries, you may find that there are many values of HDI for each country. Trimming these values by keeping only the most recent will mitigate confusing results (e.g. labeling all countries as having a small population since all countries at some point had a smaller population) and reduce the number of candidate labels we create in later steps.

In [1]:
import os
import pandas as pd
import numpy as np
from utility import run_command
from utility import rename_cols_and_overwrite_id

### Parameters
**Required**   
*quantity_file*: file path for the file that contains entity to quantity-type values  
*qualifiers_file*: file path for the file that contains wikidata labels  
*out_file*: file path for the file that we will save the trimmed results in  
*work_dir*: path to folder where intermediate files created by this notebook should be saved.  
*store_dir*: path to folder containing the sqlite3.db file that we will use for our queries. We will reuse an existing file if there is one in this folder. Otherwise we will create a new one.

In [2]:
# **REQUIRED**
quantity_file = "../../wikidata_politicians/data/claims.quantity.tsv.gz"
qualifiers_file = "../../wikidata_politicians/data/qualifiers.quantity.tsv.gz"
out_file = "../../wikidata_politicians/data/claims.quantity_trimmed.tsv.gz"
work_dir = "../../wikidata_politicians/profiler_work"
store_dir = "../../wikidata_politicians"

### Process parameters and set up variables / file names

In [3]:
# Ensure paths are absolute
quantity_file = os.path.abspath(quantity_file)
qualifiers_file = os.path.abspath(qualifiers_file)
out_file = os.path.abspath(out_file)
work_dir = os.path.abspath(work_dir)
store_dir = os.path.abspath(store_dir)
    
# Create directories
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
work_dir = "{}/trim_quantity_work".format(work_dir)
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

# Environment variables for kgtk commands
os.environ['QUANTITY_FILE'] = quantity_file
os.environ['QUALIFIERS_FILE'] = qualifiers_file
os.environ['WORK'] = work_dir
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(store_dir)
os.environ['OUT_FILE'] = out_file
os.environ['kgtk'] = "kgtk" # Need to do this for kgtk to be recognized as a command when passing it through a subprocess call

See contents of qualifiers and quantity files below:

In [4]:
!zcat $QUALIFIERS_FILE | head -5 | column -t -s $'\t'

id                                            node1                          label  node2
P1004-P4876-b5e617-af81a4c5-0-P585-d93eb6-0   P1004-P4876-b5e617-af81a4c5-0  P585   ^2019-09-11T00:00:00Z/11
P1014-P4876-605b34-fbb0b790-0-P585-cb055b-0   P1014-P4876-605b34-fbb0b790-0  P585   ^2020-09-11T00:00:00Z/11
P1014-P4876-62c45c-db5d6620-0-P585-800764-0   P1014-P4876-62c45c-db5d6620-0  P585   ^2019-12-19T00:00:00Z/11
P1014-P4876-62c45c-db5d6620-0-P642-Q151885-0  P1014-P4876-62c45c-db5d6620-0  P642   Q151885

gzip: stdout: Broken pipe


In [5]:
!zcat $QUANTITY_FILE | head -5 | column -t -s $'\t'

id                             node1  label  node2   node2;wikidatatype
P1004-P4876-b5e617-af81a4c5-0  P1004  P4876  +37427  quantity
P1014-P4876-605b34-fbb0b790-0  P1014  P4876  +53249  quantity
P1014-P4876-62c45c-db5d6620-0  P1014  P4876  +50825  quantity
P1014-P4876-732e1a-9ea8b31a-0  P1014  P4876  +47267  quantity

gzip: stdout: Broken pipe


Now using the quantity ID column and qualifiers node1 column to match quantity edges with P585 ('point in time') qualifiers.

In [6]:
!kgtk query --graph-cache $STORE \
-i $QUANTITY_FILE -i $QUALIFIERS_FILE -o $WORK/quantities_with_qualifiers.tsv.gz \
--match '`'"$QUANTITY_FILE"'`: (n1)-[l {label:prop}]->(n2), `'"$QUALIFIERS_FILE"'`: (l)-[q {label:P585}]->(t)' \
--return 'distinct n1, prop as label, n2 as node2, q.label as qualifier, kgtk_date_and_time(t) as time, l as id' \
--order-by 'n1, prop, q.label, time desc'

In [7]:
!zcat $WORK/quantities_with_qualifiers.tsv.gz | head | column -t -s $'\t'

node1  label  node2      qualifier  time                   id
P1004  P4876  +37427     P585       ^2019-09-11T00:00:00Z  P1004-P4876-b5e617-af81a4c5-0
P1014  P4876  +53249     P585       ^2020-09-11T00:00:00Z  P1014-P4876-605b34-fbb0b790-0
P1014  P4876  +50825     P585       ^2019-12-19T00:00:00Z  P1014-P4876-62c45c-db5d6620-0
P1014  P4876  +46591     P585       ^2019-07-18T00:00:00Z  P1014-P4876-f7a212-293bd556-0
P1014  P4876  +47267     P585       ^2018-06-17T00:00:00Z  P1014-P4876-732e1a-9ea8b31a-0
P1014  P4876  +50825     P642                              P1014-P4876-62c45c-db5d6620-0
P1022  P4876  +541       P585       ^2019-08-30T00:00:00Z  P1022-P4876-fa0ca8-40e422f8-0
P1042  P1114  +1952404   P585       ^2020-02-13T00:00:00Z  P1042-P1114-27b781-00b6c31d-0
P1044  P4876  +54300000  P585       ^2019-00-00T00:00:00Z  P1044-P4876-d72e5b-569d2ef4-0

gzip: stdout: Broken pipe


Next, remove the most recent edge for each <node1, label> pair

In [8]:
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    return result
df = pd.read_csv("{}/quantities_with_qualifiers.tsv.gz".format(os.environ["WORK"]), delimiter = '\t').fillna("")
mask = df.groupby(['node1','label'])["node1"].transform(mask_first).astype(bool)
out_file = "{}/quantity_edges_to_filter.tsv.gz".format(os.environ["WORK"])
df.loc[mask].to_csv(out_file, sep='\t', index = False)

See that the first edge for node1=Q1000, label=P1081 has been removed in this new file:

In [9]:
!zcat $WORK/quantity_edges_to_filter.tsv.gz | head | column -t -s $'\t'

node1  label  node2     qualifier  time                   id
P1014  P4876  +50825    P585       ^2019-12-19T00:00:00Z  P1014-P4876-62c45c-db5d6620-0
P1014  P4876  +46591    P585       ^2019-07-18T00:00:00Z  P1014-P4876-f7a212-293bd556-0
P1014  P4876  +47267    P585       ^2018-06-17T00:00:00Z  P1014-P4876-732e1a-9ea8b31a-0
P1014  P4876  +50825    P642                              P1014-P4876-62c45c-db5d6620-0
P1185  P4876  +1251267  P585       ^2020-03-19T00:00:00Z  P1185-P4876-03ae44-80497f15-0
P1278  P4876  +1217727  P585       ^2018-06-28T00:00:00Z  P1278-P4876-a0a477-23b4d7c5-0
P1667  P4876  +2505678  P585       ^2018-10-05T00:00:00Z  P1667-P4876-fd5e17-6879f084-0
P1667  P4876  +2505616  P585       ^2018-06-17T00:00:00Z  P1667-P4876-fecd0c-099cd46e-0
P1838  P4876  +61158    P585       ^2019-04-19T00:00:00Z  P1838-P4876-12bbb7-aca29bf1-0

gzip: stdout: Broken pipe


Now remove edges from the original quantity file if they are present in the edges-to-filter file. This will give us the final trimmed quantity file.

In [10]:
!kgtk ifnotexists -i $QUANTITY_FILE --filter-on $WORK/quantity_edges_to_filter.tsv.gz -o $OUT_FILE \
--input-keys id --filter-keys id

In [11]:
!zcat $OUT_FILE | head | column -t -s $'\t'

id                             node1  label  node2      node2;wikidatatype
P1004-P4876-b5e617-af81a4c5-0  P1004  P4876  +37427     quantity
P1014-P4876-605b34-fbb0b790-0  P1014  P4876  +53249     quantity
P1022-P4876-fa0ca8-40e422f8-0  P1022  P4876  +541       quantity
P1042-P1114-27b781-00b6c31d-0  P1042  P1114  +1952404   quantity
P1044-P4876-d72e5b-569d2ef4-0  P1044  P4876  +54300000  quantity
P1051-P4876-32013b-1caeb73e-0  P1051  P4876  +14157     quantity
P1182-P4876-d9b3ca-4cf79593-0  P1182  P4876  +6500000   quantity
P1187-P4876-ba6287-1f8f2372-0  P1187  P4876  +42496     quantity
P1188-P4876-3fd83d-fd17e76d-0  P1188  P4876  +58916     quantity

gzip: stdout: Broken pipe


Number of edges in original file:

In [12]:
!zcat $QUANTITY_FILE | wc -l

591274


Number of edges in trimmed file:

In [13]:
!zcat $OUT_FILE | wc -l

87874
